<a href="https://colab.research.google.com/github/jvrupp/NFTbot/blob/main/SistemaDeCaptacaoDeDadosTempoReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O sistema funciona , mas ainda precisa ter uma lapidaçao na parte de captaçao de restos


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests, re
import time
import json
import pandas as pd
banco_dados=[]
banco_dados2=[]
definitive_data={}

from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox") # linux only
chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works

driver = webdriver.Chrome(executable_path=r'./chromedriver.exe',options=chrome_options)





def minera_paginas(paginas_restantes,last):
    jp=0#paginas ja adquridas
    driver.get('https://webapi.mir4global.com/exd/lists?listType=recent&languageCode=en&page=1&class=0&itemType=0&grade=0&tier=0&enhance=-1&minPrice=0&maxPrice=0&uniqueFlag=0')
    codigo_temp=driver.find_element_by_tag_name('pre')
    json_string = """{}""".format(codigo_temp.text)
    stud_obj = json.loads(json_string)
    total_paginas=(stud_obj['data']['totalCount'] - last)//10#total paginas em tempo real
    for i in range(1,(paginas_restantes)+1):
        varContr=total_paginas-paginas_restantes
        
        controle = 1 + varContr + jp
        driver.get('https://webapi.mir4global.com/exd/lists?listType=recent&languageCode=en&page={}&class=0&itemType=0&grade=0&tier=0&enhance=-1&minPrice=0&maxPrice=0&uniqueFlag=0'.format(controle))
        codigo_temp=driver.find_element_by_tag_name('pre')
        json_string = """{}""".format(codigo_temp.text)
        stud_obj = json.loads(json_string)
        total_paginas=(stud_obj['data']['totalCount'] - last)//10
        for n in range(0,len(stud_obj['data']['lists'])):
            print('minerando paginas')
            print(stud_obj['data']['lists'][n]['item']['itemName'],float(stud_obj['data']['lists'][n]['price'])*18,stud_obj['data']['lists'][n]['endTime'],stud_obj['data']['totalCount'] - (jp*10))
        jp = jp + 1
    
    return (jp,controle)
        
        
        

def minera_restos(n_paginas,linhas_restantes_pag):
    linhas_restantes,paginas_restantes=linhas_restantes_pag
    driver.get('https://webapi.mir4global.com/exd/lists?listType=recent&languageCode=en&page=1&class=0&itemType=0&grade=0&tier=0&enhance=-1&minPrice=0&maxPrice=0&uniqueFlag=0')
    codigo_temp=driver.find_element_by_tag_name('pre')
    json_string = """{}""".format(codigo_temp.text)
    stud_obj = json.loads(json_string)
    total_paginas=(stud_obj['data']['totalCount'] - last)//10

    varContr=total_paginas-paginas_restantes
    pag =  1 + n_paginas + varContr
    driver.get('https://webapi.mir4global.com/exd/lists?listType=recent&languageCode=en&page={}&class=0&itemType=0&grade=0&tier=0&enhance=-1&minPrice=0&maxPrice=0&uniqueFlag=0'.format(pag))
    codigo_temp=driver.find_element_by_tag_name('pre')
    json_string = """{}""".format(codigo_temp.text)
    stud_obj = json.loads(json_string)
    for n in range(0,linhas_restantes):#prestar atencao aqui
        print('minerando restos')
        print(stud_obj['data']['lists'][n]['item']['itemName'],float(stud_obj['data']['lists'][n]['price'])*18,stud_obj['data']['lists'][n]['endTime'])
    return linhas_restantes
    
    
def minera_ambos(paginas_restantes,linhas_restantes,last):
    
    jp,prox_rest=minera_paginas(paginas_restantes,last)
    pag=minera_restos(prox_rest,(linhas_restantes,paginas_restantes))
    return jp ,pag
    
paginas_restantes= 0
linhas_restantes = 0
last = 11200 #total de linhas que se tem

while True:
    driver.get('https://webapi.mir4global.com/exd/lists?listType=recent&languageCode=en&page=1&class=0&itemType=0&grade=0&tier=0&enhance=-1&minPrice=0&maxPrice=0&uniqueFlag=0')
    
    codigo_temp=driver.find_element_by_tag_name('pre')
    json_string = """{}""".format(codigo_temp.text)
    stud_obj = json.loads(json_string)
    paginas_restantes = (stud_obj['data']['totalCount'] - last)//10
    linhas_restantes = (stud_obj['data']['totalCount'] - last)%10
    if paginas_restantes !=0 or linhas_restantes!=0 :#verifica deficit de dados
        if paginas_restantes > 0  and  linhas_restantes == 0:
           print('Minera paginas precisando de {} pag '.format(paginas_restantes))
           jp,prox_rest=minera_paginas(paginas_restantes,last)
           last = last + (jp*10)#last é o que eu peguei
           jp=0
        elif paginas_restantes == 0  and  linhas_restantes > 0:
            print('Minera linhas precisando de  {} restos'.format(linhas_restantes))
            #quando caio aqui dentro significa que esotu na ultima pagina disponivel
            pag=minera_restos(paginas_restantes,(linhas_restantes,0))#retorna a pagina que o resto esta
            last = last + pag
            jp=0#last é o que eu peguei
        elif paginas_restantes > 0  and  linhas_restantes > 0:
            print('Minera ambos precisando de {} pag e {} restos'.format(paginas_restantes,linhas_restantes))
            jp,pag=minera_ambos(paginas_restantes, linhas_restantes, last)
            last = last + ((jp*10)+pag)#last é o que eu peguei
            jp=0
        else:
            print('nao encontramos nenhum caso')
    print(stud_obj['data']['totalCount'],last)